In [1]:
import os
import pandas as pd
import json
import numpy as np

In [2]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [3]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA['NUMBER_OF_FEATURES']

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

LABEL_COLUMN = 'Class_MotionSpeed'

# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\\TEST2\\'

# save numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TEST_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'
ALL_Y_TEST_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'

In [4]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'SLOW': 0, 'AVERAGE': 1, 'FAST': 2}


In [5]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and "SML" in file and "STEPS" in file and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TEST2-SIDESTEPS-L-SML-100BPM-AUGMENT.xlsx', 'PROC-TEST2-SIDESTEPS-L-SML-25BPM-AUGMENT.xlsx', 'PROC-TEST2-SIDESTEPS-L-SML-50BPM.xlsx', 'PROC-TEST2-SIDESTEPS-R-SML-100BPM-AUGMENT.xlsx', 'PROC-TEST2-SIDESTEPS-R-SML-25BPM-AUGMENT.xlsx', 'PROC-TEST2-SIDESTEPS-R-SML-50BPM.xlsx', 'PROC-TEST2-STEPS-LR-SML-100BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-130BPM.xlsx', 'PROC-TEST2-STEPS-LR-SML-180BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-25BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-260BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-45BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-50BPM.xlsx', 'PROC-TEST2-STEPS-LR-SML-65BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-90BPM.xlsx']


In [6]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 180)/360

In [7]:
def df2Xy(df, windowSize=5):
  X = []
  y = []

  # NORMALIZE THE SENSOR DATA
  #df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalizeSensorData)
  #df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalizeSensorData)
  #df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalizeSensorData)
  #df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalizeSensorData)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()

        # add row values to the input
        input_data_list.append(row_values)

    # turnn list into array to do arthimetic
    input_data_array = np.array(input_data_list)

    # remove all negative sinces we want to the total change (we dont care which direction)
    input_data_array = np.abs(input_data_array)

    # calculate the total change for each sennsor value (TOTAL POSITIVE SENSOR CHANGES)
    input_data_array = np.sum(input_data_array, axis=0)

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), [LABEL_COLUMN]].values.tolist()

      y.append(label)

  return (np.array(X), np.array(y))

In [8]:
ALL_X = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y = np.empty((0))  # List to store all Y training data

In [9]:
def processData(ALL_X, ALL_Y):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTest, yTest = df2Xy(df, WINDOW_SIZE)

        yTest = yTest.reshape(-1)
        
        ALL_X = np.concatenate([ALL_X, xTest])
        ALL_Y = np.concatenate([ALL_Y, yTest])
        
        print('xTest.shape:', xTest.shape)
        print('ALL_X.shape:', ALL_X.shape, ' ALL_Y.shape:', ALL_Y.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y)
    

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TEST_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TEST_CLASSIFY_PATH, numerical_label)


In [10]:
if not os.path.exists(ALL_X_TEST_CLASSIFY_PATH):
    processData(ALL_X, ALL_Y)

ALL_X_TEST_CLASSIFY = np.load(ALL_X_TEST_CLASSIFY_PATH)
ALL_Y_TEST_CLASSIFY = np.load(ALL_Y_TEST_CLASSIFY_PATH)

print(ALL_X_TEST_CLASSIFY.shape)
print(ALL_Y_TEST_CLASSIFY.shape)

xTest.shape: (3852, 4)
ALL_X.shape: (3852, 4)  ALL_Y.shape: (3852,)
-------------------------------
xTest.shape: (7724, 4)
ALL_X.shape: (11576, 4)  ALL_Y.shape: (11576,)
-------------------------------
xTest.shape: (3853, 4)
ALL_X.shape: (15429, 4)  ALL_Y.shape: (15429,)
-------------------------------
xTest.shape: (3590, 4)
ALL_X.shape: (19019, 4)  ALL_Y.shape: (19019,)
-------------------------------
xTest.shape: (7200, 4)
ALL_X.shape: (26219, 4)  ALL_Y.shape: (26219,)
-------------------------------
xTest.shape: (3591, 4)
ALL_X.shape: (29810, 4)  ALL_Y.shape: (29810,)
-------------------------------
xTest.shape: (4929, 4)
ALL_X.shape: (34739, 4)  ALL_Y.shape: (34739,)
-------------------------------
xTest.shape: (3776, 4)
ALL_X.shape: (38515, 4)  ALL_Y.shape: (38515,)
-------------------------------
xTest.shape: (4548, 4)
ALL_X.shape: (43063, 4)  ALL_Y.shape: (43063,)
-------------------------------
xTest.shape: (9878, 4)
ALL_X.shape: (52941, 4)  ALL_Y.shape: (52941,)
--------------

In [11]:
print(np.max(ALL_X_TEST_CLASSIFY))

135.0
